# Etude consommation moyenne brute par heure de gaz et d'électricité

La base de données provient de l'URL: 
https://www.data.gouv.fr/fr/datasets/consommation-quotidienne-brute/

Petit descriptif de la database demi-heure par demi-heure: 
- consommation ken éléctricité enn MW
- consommation de gaz en MW

Information sur le nucléaire français:
https://www.edf.fr/groupe-edf/espaces-dedies/l-energie-de-a-a-z/tout-sur-l-energie/produire-de-l-electricite/le-nucleaire-en-chiffres

In [1]:
import pandas as pd 
import seaborn as sns 
import time 

from datetime import datetime

In [2]:
raw_data = pd.read_csv("./raw_daily_conso.csv", sep=";")
print(raw_data.shape)
raw_data.head(10)

(192863, 11)


,date_heure,date,heure,consommation_brute_gaz_grtgaz,statut_grtgaz,consommation_brute_gaz_terega,statut_terega,consommation_brute_gaz_totale,consommation_brute_electricite_rte,statut_rte,consommation_brute_totale
0,2022-12-31T23:30:00+01:00,31/12/2022,23:30,NaN,NaN,NaN,NaN,NaN,NaN,Consolidé,NaN
1,2022-12-31T23:00:00+01:00,31/12/2022,23:00,NaN,Définitif,NaN,Définitif,NaN,NaN,Consolidé,NaN
2,2022-12-31T22:30:00+01:00,31/12/2022,22:30,NaN,NaN,NaN,NaN,NaN,NaN,Consolidé,NaN
3,2022-12-31T22:00:00+01:00,31/12/2022,22:00,NaN,Définitif,NaN,Définitif,NaN,NaN,Consolidé,NaN
4,2022-12-31T21:30:00+01:00,31/12/2022,21:30,NaN,NaN,NaN,NaN,NaN,NaN,Consolidé,NaN
5,2022-12-31T21:00:00+01:00,31/12/2022,21:00,NaN,Définitif,NaN,Définitif,NaN,NaN,Consolidé,NaN
6,2022-12-31T20:30:00+01:00,31/12/2022,20:30,NaN,NaN,NaN,NaN,NaN,NaN,Consolidé,NaN
7,2022-12-31T20:00:00+01:00,31/12/2022,20:00,NaN,Définitif,NaN,Définitif,NaN,NaN,Consolidé,NaN
8,2022-12-31T19:30:00+01:00,31/12/2022,19:30,NaN,NaN,NaN,NaN,NaN,NaN,Consolidé,NaN
9,2022-12-31T19:00:00+01:00,31/12/2022,19:00,NaN,Définitif,NaN,Définitif,NaN,NaN,Consolidé,NaN


In [3]:
# drop rows too recent that does not contain data 
index_max = raw_data.loc[raw_data.date == '01/10/2022'].index.max()
i_to_drop = [i for i in range(index_max)]
raw_data.drop(labels=i_to_drop, axis=0, inplace=True)

In [4]:
miss_index_conso_elec = list(raw_data
                             .loc[raw_data
                                  .consommation_brute_electricite_rte
                                  .isna()]
                             .index)

all = set([i for i in range(raw_data.index.min(), max(miss_index_conso_elec)+1)])
all_ = set(miss_index_conso_elec)

if len(all - all_) == 0: 
    print("Les premières lignes du dataset ne contiennet pas les données sur la consommation brute électrique")
    date = raw_data.at[max(miss_index_conso_elec)+1, 'date']
    print(f"Le datset est complet (données électricité + gaz) jusqu'au {date}")

Les premières lignes du dataset ne contiennet pas les données sur la consommation brute électrique
Le datset est complet (données électricité + gaz) jusqu'au 31/05/2022


In [5]:
inc_rows = [i for i in range(raw_data.index.min(), max(miss_index_conso_elec)+1)]

comp_data = raw_data.drop(labels=inc_rows, axis=0)

comp_data.head(4)

,date_heure,date,heure,consommation_brute_gaz_grtgaz,statut_grtgaz,consommation_brute_gaz_terega,statut_terega,consommation_brute_gaz_totale,consommation_brute_electricite_rte,statut_rte,consommation_brute_totale
10271,2022-05-31T23:30:00+02:00,31/05/2022,23:30,NaN,NaN,NaN,NaN,NaN,45279.0,Consolidé,NaN
10272,2022-05-31T23:00:00+02:00,31/05/2022,23:00,27364.0,Définitif,1164.0,Définitif,28528.0,46703.0,Consolidé,75231.0
10273,2022-05-31T22:30:00+02:00,31/05/2022,22:30,NaN,NaN,NaN,NaN,NaN,45067.0,Consolidé,NaN
10274,2022-05-31T22:00:00+02:00,31/05/2022,22:00,28613.0,Définitif,1218.0,Définitif,29831.0,44019.0,Consolidé,73850.0


In [6]:
# drop column date_heure (information already contained into columns date and heure)
comp_data.drop(columns=['date_heure'], inplace=True)

In [7]:
# correct the row that is present twice
comp_data.at[36334, 'date']='06/12/2020'
comp_data['date'] = pd.to_datetime(comp_data['date'], dayfirst=True)
comp_data['heure'] = pd.to_datetime(comp_data['heure'], format='%H:%M') - pd.to_datetime(comp_data['heure'], format='%H:%M').dt.normalize()

comp_data.head()


,date,heure,consommation_brute_gaz_grtgaz,statut_grtgaz,consommation_brute_gaz_terega,statut_terega,consommation_brute_gaz_totale,consommation_brute_electricite_rte,statut_rte,consommation_brute_totale
10271,2022-05-31,0 days 23:30:00,NaN,NaN,NaN,NaN,NaN,45279.0,Consolidé,NaN
10272,2022-05-31,0 days 23:00:00,27364.0,Définitif,1164.0,Définitif,28528.0,46703.0,Consolidé,75231.0
10273,2022-05-31,0 days 22:30:00,NaN,NaN,NaN,NaN,NaN,45067.0,Consolidé,NaN
10274,2022-05-31,0 days 22:00:00,28613.0,Définitif,1218.0,Définitif,29831.0,44019.0,Consolidé,73850.0
10275,2022-05-31,0 days 21:30:00,NaN,NaN,NaN,NaN,NaN,44043.0,Consolidé,NaN


In [9]:
comp_data.at[36334, 'date']='2020-12-06'
print(comp_data.loc[comp_data.index == 36334])
comp_data.sort_values(by=['date', 'heure'], inplace=True)
comp_data.reset_index(inplace=True, drop=True)

            date  heure  consommation_brute_gaz_grtgaz statut_grtgaz  \
36334 2020-12-06 0 days                        68843.0     Définitif   

       consommation_brute_gaz_terega statut_terega  \
36334                         4275.0     Définitif   

       consommation_brute_gaz_totale  consommation_brute_electricite_rte  \
36334                        73118.0                             64891.0   

      statut_rte  consommation_brute_totale  
36334  Consolidé                   138009.0  


In [10]:
# check if there is odd index with not corresponding hours like **:00 (pb for cleaninig data)
len(pd.unique(comp_data.loc[comp_data.index % 2 == 1].heure))

24

In [ ]:
grouped = comp_data.groupby(by=['date', 'heure'])
grouped.filter(lambda x: x['consommation_brute_electricite_rte'].count() > 1)

,date,heure,consommation_brute_gaz_grtgaz,statut_grtgaz,consommation_brute_gaz_terega,statut_terega,consommation_brute_gaz_totale,consommation_brute_electricite_rte,statut_rte,consommation_brute_totale
156576,2020-12-06,0 days 00:30:00,NaN,NaN,NaN,NaN,NaN,63446.0,Consolidé,NaN
156577,2020-12-06,0 days 00:30:00,NaN,NaN,NaN,NaN,NaN,42742.0,Consolidé,NaN


In [ ]:
col = ['date', 'heure', 'consommation_brute_gaz_totale']

print(comp_data[col].loc[(comp_data.index >= 36280)
                         & (comp_data.index <= 36292)])

print(comp_data[col].loc[(comp_data.index >= 36330)
                         & (comp_data.index <= 36345)])

            date           heure  consommation_brute_gaz_totale
36280 2020-05-06 0 days 03:30:00                            NaN
36281 2020-05-06 0 days 03:00:00                        30392.0
36282 2020-05-06 0 days 02:30:00                            NaN
36283 2020-05-06 0 days 02:00:00                        29841.0
36284 2020-05-06 0 days 01:30:00                            NaN
36285 2020-05-06 0 days 01:00:00                        29390.0
36286 2020-05-06 0 days 00:30:00                            NaN
36287 2020-05-06 0 days 00:00:00                        29930.0
36288 2020-05-05 0 days 23:30:00                            NaN
36289 2020-05-05 0 days 23:00:00                        30662.0
36290 2020-05-05 0 days 22:30:00                            NaN
36291 2020-05-05 0 days 22:00:00                        31708.0
36292 2020-05-05 0 days 21:30:00                            NaN
            date           heure  consommation_brute_gaz_totale
36330 2020-05-05 0 days 02:30:00        

In [ ]:
# Maintenant, avoir les données demi-heure par demi-heure pour l'électricité ne nous arrange pas 
# Nous allons donc combiner les deux colonnnes pour produire un dataset heure par heure uniquement

def transformation_1(row):
    if row.name % 2 == 1:
        return row.consommation_brute_electricite_rte
    i = row.name
    to_add = comp_data.at[i+1, 'consommation_brute_electricite_rte'] 
    conso_hph = (row.consommation_brute_electricite_rte + to_add) / 2
    return conso_hph

comp_data.drop(labels=raw_data.index.max(), inplace=True)

comp_data['consommation_brute_electricite_rte_hph'] = comp_data.apply(func=transformation_1, axis=1)

columns_to_show = ['date', 'heure', 'consommation_brute_electricite_rte', 'consommation_brute_electricite_rte_hph']
comp_data[columns_to_show].head(10)

,date,heure,consommation_brute_electricite_rte,consommation_brute_electricite_rte_hph
10271,31/05/2022,23:30,45279.0,45279.0
10272,31/05/2022,23:00,46703.0,45885.0
10273,31/05/2022,22:30,45067.0,45067.0
10274,31/05/2022,22:00,44019.0,44031.0
10275,31/05/2022,21:30,44043.0,44043.0
10276,31/05/2022,21:00,44677.0,45287.5
10277,31/05/2022,20:30,45898.0,45898.0
10278,31/05/2022,20:00,47317.0,47712.0
10279,31/05/2022,19:30,48107.0,48107.0
10280,31/05/2022,19:00,48330.0,47756.5


In [ ]:
labels_to_drop = [i
                  for i in range(comp_data.index.min(), comp_data.index.max()+1)
                  if i % 2 == 1]
data_hh = comp_data.drop(labels=labels_to_drop, axis=0)
print(f"Shape of the data_hh dataframe is {data_hh.shape}")
data_hh.head()

Shape of the data_hh dataframe is (91295, 11)


,date,heure,consommation_brute_gaz_grtgaz,statut_grtgaz,consommation_brute_gaz_terega,statut_terega,consommation_brute_gaz_totale,consommation_brute_electricite_rte,statut_rte,consommation_brute_totale,consommation_brute_electricite_rte_hph
10272,31/05/2022,23:00,27364.0,Définitif,1164.0,Définitif,28528.0,46703.0,Consolidé,75231.0,45885.0
10274,31/05/2022,22:00,28613.0,Définitif,1218.0,Définitif,29831.0,44019.0,Consolidé,73850.0,44031.0
10276,31/05/2022,21:00,29989.0,Définitif,1328.0,Définitif,31317.0,44677.0,Consolidé,75994.0,45287.5
10278,31/05/2022,20:00,30827.0,Définitif,1470.0,Définitif,32297.0,47317.0,Consolidé,79614.0,47712.0
10280,31/05/2022,19:00,30721.0,Définitif,1407.0,Définitif,32128.0,48330.0,Consolidé,80458.0,47756.5


In [ ]:
# maintenant nous devons mettre à jour la consommation brute totale heure par heure

def transformation_2(row):
    gaz = row.consommation_brute_gaz_totale
    electricite = row.consommation_brute_electricite_rte_hph
    return gaz + electricite

data_hh['consommation_brute_totale_hph'] = data_hh.apply(func=transformation_2, 
                                                         axis=1)

columns_to_show.append('consommation_brute_totale_hph')
data_hh[columns_to_show].head()

,date,heure,consommation_brute_electricite_rte,consommation_brute_electricite_rte_hph,consommation_brute_totale_hph
10272,31/05/2022,23:00,46703.0,45885.0,74413.0
10274,31/05/2022,22:00,44019.0,44031.0,73862.0
10276,31/05/2022,21:00,44677.0,45287.5,76604.5
10278,31/05/2022,20:00,47317.0,47712.0,80009.0
10280,31/05/2022,19:00,48330.0,47756.5,79884.5


In [ ]:
# maintenant nous voulons nous débarasser de toutes les colonnes qui ne nous serviront pas 
columns_to_drop = ['consommation_brute_electricite_rte', 
                   'consommation_brute_totale']
data_hh.drop(columns=columns_to_drop, inplace=True)
data_hh.head()

,date,heure,consommation_brute_gaz_grtgaz,statut_grtgaz,consommation_brute_gaz_terega,statut_terega,consommation_brute_gaz_totale,statut_rte,consommation_brute_electricite_rte_hph,consommation_brute_totale_hph
10272,31/05/2022,23:00,27364.0,Définitif,1164.0,Définitif,28528.0,Consolidé,45885.0,74413.0
10274,31/05/2022,22:00,28613.0,Définitif,1218.0,Définitif,29831.0,Consolidé,44031.0,73862.0
10276,31/05/2022,21:00,29989.0,Définitif,1328.0,Définitif,31317.0,Consolidé,45287.5,76604.5
10278,31/05/2022,20:00,30827.0,Définitif,1470.0,Définitif,32297.0,Consolidé,47712.0,80009.0
10280,31/05/2022,19:00,30721.0,Définitif,1407.0,Définitif,32128.0,Consolidé,47756.5,79884.5


In [ ]:
# nous voulons meme construire un dataframe sans le status
datalight_hh = data_hh.drop(columns=['statut_terega', 'statut_grtgaz', 'statut_rte'])
datalight_hh.head()

,date,heure,consommation_brute_gaz_grtgaz,consommation_brute_gaz_terega,consommation_brute_gaz_totale,consommation_brute_electricite_rte_hph,consommation_brute_totale_hph
10272,31/05/2022,23:00,27364.0,1164.0,28528.0,45885.0,74413.0
10274,31/05/2022,22:00,28613.0,1218.0,29831.0,44031.0,73862.0
10276,31/05/2022,21:00,29989.0,1328.0,31317.0,45287.5,76604.5
10278,31/05/2022,20:00,30827.0,1470.0,32297.0,47712.0,80009.0
10280,31/05/2022,19:00,30721.0,1407.0,32128.0,47756.5,79884.5


In [ ]:
# nouveaux index depuis 0 
datalight_hh.reset_index(inplace=True, drop=True)
datalight_hh.head()

,date,heure,consommation_brute_gaz_grtgaz,consommation_brute_gaz_terega,consommation_brute_gaz_totale,consommation_brute_electricite_rte_hph,consommation_brute_totale_hph
0,31/05/2022,23:00,27364.0,1164.0,28528.0,45885.0,74413.0
1,31/05/2022,22:00,28613.0,1218.0,29831.0,44031.0,73862.0
2,31/05/2022,21:00,29989.0,1328.0,31317.0,45287.5,76604.5
3,31/05/2022,20:00,30827.0,1470.0,32297.0,47712.0,80009.0
4,31/05/2022,19:00,30721.0,1407.0,32128.0,47756.5,79884.5


In [ ]:
columns_to_describe = ['consommation_brute_gaz_totale', 
                       'consommation_brute_electricite_rte_hph',
                       'consommation_brute_totale_hph' ]

datalight_hh[columns_to_describe].describe()

,consommation_brute_gaz_totale,consommation_brute_electricite_rte_hph,consommation_brute_totale_hph
count,91256.000000,91295.000000,91256.000000
mean,54009.456178,54144.119946,108151.059366
std,29004.781911,12003.281026,40054.904780
min,3670.000000,28758.000000,45433.000000
25%,28508.000000,44964.500000,74743.375000
50%,47027.000000,52498.000000,97820.500000
75%,75982.500000,62413.500000,138105.250000
max,176040.000000,100516.000000,274473.000000


In [ ]:
# maintenant, nous aimerions pouvoir changer les types des colonnes : 
datalight_hh['date'] = pd.to_datetime(datalight_hh['date'], dayfirst=True)

In [ ]:
datalight_hh.shape

(91295, 7)